## Gibb's Sampling for the Object Detection Example

In [6]:
import random
from collections import Counter
import math

# Define weights for the states
weights = {
    (0, 1, 1): 4,
    (0, 1, 2): 4,
    (1, 1, 1): 4,
    (1, 1, 2): 4,
    (1, 2, 1): 2,
    (1, 2, 2): 8
}

# Initialize state vector randomly
current_state = [random.choice([0, 1, 2]) for _ in range(3)]
print(current_state)

# Number of iterations
num_iterations = 1000000

# Initialize counters for marginal probabilities
counts = Counter()

[1, 1, 2]


In [7]:
# Sampling function
def sample_variable(index, state):
    # Generate the conditional distribution for the variable
    conditional_weights = []
    for value in [0, 1, 2]:
        new_state = state[:]
        new_state[index] = value
        weight = weights.get(tuple(new_state), 0)
        conditional_weights.append((value, weight))

    # Normalize weights to create a probability distribution
    total_weight = sum(w for _, w in conditional_weights)
    if total_weight is None or math.isnan(total_weight) or total_weight == 0:
        # Handle the zero or NA weight case: assign probabilities of 0
        probabilities = [(v, 0) for v, _ in conditional_weights]
    else:
        probabilities = [(v, w / total_weight) for v, w in conditional_weights]

    # Sample from the conditional distribution
    r = random.random()
    cumulative = 0
    for value, prob in probabilities:
        cumulative += prob
        if r < cumulative:
            return value
    return probabilities[-1][0]  # Fallback in case of rounding errors

- _Weights_: The weights dictionary specifies the given weights for each state.
- _Initialization_: The state vector is initialized randomly.
- _Sampling_:For each variable, we compute a conditional distribution using 
  weights for all possible values of that variable while keeping other 
  variables fixed.

In [8]:
# Gibbs sampling
for _ in range(num_iterations):
    for i in range(3):  # Iterate over each variable x1, x2, x3
        current_state[i] = sample_variable(i, current_state)
    counts[tuple(current_state)] += 1

- _Gibbs Sampling_: The algorithm iterates over variables and samples a new value 
  for each variable based on its conditional distribution.
- _Counting_: Counts for state occurrences and marginal probabilities are computed.

In [9]:
# Compute estimated marginal probabilities
marginal_counts = Counter()
for state, count in counts.items():
    for i, value in enumerate(state):
        marginal_counts[(i, value)] += count

In [10]:
marginal_probabilities = {
    (i, v): marginal_counts[(i, v)] / num_iterations for i in range(3) for v in [0, 1, 2]
}

# Output results
print("Counts for each variable and value:")
for (i, v), count in marginal_counts.items():
    print(f"X_{i+1}={v}: {count}")

print("\nEstimated marginal probabilities:")
for (i, v), prob in marginal_probabilities.items():
    print(f"P(X_{i+1}={v}): {prob:.3f}")

Counts for each variable and value:
X_1=1: 691158
X_2=2: 383432
X_3=2: 615270
X_2=1: 616568
X_3=1: 384730
X_1=0: 308842

Estimated marginal probabilities:
P(X_1=0): 0.309
P(X_1=1): 0.691
P(X_1=2): 0.000
P(X_2=0): 0.000
P(X_2=1): 0.617
P(X_2=2): 0.383
P(X_3=0): 0.000
P(X_3=1): 0.385
P(X_3=2): 0.615


The __true__ marginal probabilities for $X_2=1$ is 0.615 and for $X_2=2$.